In [22]:
!python bio-medrxiv.py testeins -f /data/Goldlabel_biomedRxiv/goldlabel1_docs801-840_laura/goldlabel1_tasknameshort_MedRxiv_docs/ -c -M -C -L

Traceback (most recent call last):
  File "bio-medrxiv.py", line 103, in <module>
    plac.call(main)
  File "/usr/local/lib/python3.8/site-packages/plac_core.py", line 367, in call
    cmd, result = parser.consume(arglist)
  File "/usr/local/lib/python3.8/site-packages/plac_core.py", line 232, in consume
    return cmd, self.func(*(args + varargs + extraopts), **kwargs)
  File "bio-medrxiv.py", line 30, in main
    create_db(db_name)
  File "/workspaces/bio-medRxiv/MeMoKBC/src/MeMoKBC/pipeline/utils.py", line 19, in create_db
    cur.execute(sql)
psycopg2.errors.DuplicateDatabase: database "testeins" already exists



In [1]:
!python bio-medrxiv.py testeins -L

[2023-06-25 01:17:33,057][INFO] fonduer.meta:49 - Setting logging directory to: /tmp/2023-06-25_01-17-33
[2023-06-25 01:17:33,214][INFO] fonduer.supervision.labeler:358 - Clearing ALL Labels and LabelKeys.
/usr/local/lib/python3.8/site-packages/fonduer/utils/utils_udf.py:217: SAWarning: Coercing Subquery object into a select() for use in IN(); please pass a select() construct explicitly
  .filter(candidate_class.id.in_(sub_query))
[2023-06-25 01:17:33,924][INFO] fonduer.supervision.labeler:330 - Clearing Labels (split 0)
/usr/local/lib/python3.8/site-packages/fonduer/supervision/labeler.py:340: SAWarning: Coercing Subquery object into a select() for use in IN(); please pass a select() construct explicitly
  query = self.session.query(table).filter(table.candidate_id.in_(sub_query))
[2023-06-25 01:17:33,939][INFO] fonduer.utils.udf:67 - Running UDF...
100%|███████████████████████████████████████████| 14/14 [00:04<00:00,  3.27it/s]
[2023-06-25 01:17:38,615][INFO] fonduer.supervision.labe

imports

In [2]:
from fonduer.supervision import Labeler
from fonduer.features import Featurizer
from fonduer.candidates.models import Candidate

from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import LabelModel

from MeMoKBC.pipeline.utils import get_session, load_candidates, match_label_matrix
from MeMoKBC.definitions.candidates import NameFullAbbr, NameAbbrTask
from MeMoKBC.pipeline.lfs.name_short_long_lfs import short_long_lfs
from MeMoKBC.pipeline.lfs.name_short_task_lfs import name_abbr_task_lfs
from MeMoKBC.gold_label_matcher import match_gold_label
from importlib import reload
import csv

Get session object

In [3]:
session = get_session(db_name="testeins")

[2023-06-25 01:18:02,023][INFO] fonduer.meta:49 - Setting logging directory to: /tmp/2023-06-25_01-18-02


In [4]:
'''
FALSE = 0
TRUE = 1
ABSTAIN = -1

def is_medical_abbreviation(c):
    name_abbr, task = c
    name_span = name_abbr.context.get_span()
    print(name_span)

    with open('/workspaces/bio-medRxiv/data/CSVs/all_abbs.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        medical_abbreviations = [row[0].lower() for row in reader]

    if name_span.lower() in medical_abbreviations:
        return FALSE

    return ABSTAIN

SyntaxError: EOF while scanning triple-quoted string literal (2203272684.py, line 18)

In [5]:
#cands = session.query(NameAbbrTask).all()

In [6]:
#cands[10]

In [7]:
'''
i = 0
while i<10:
    output = is_medical_abbreviation(cands[i])
    print(output)
    i = i+1
'''

'\ni = 0\nwhile i<10:\n    output = is_medical_abbreviation(cands[i])\n    print(output)\n    i = i+1\n'

Define candidates and Labeler object

In [8]:
candidates = [NameAbbrTask, NameFullAbbr]

Load candidates and labels

In [9]:
L_train_NAT, L_train_NFA = match_label_matrix(session, candidates, 0)

/workspaces/bio-medRxiv/MeMoKBC/src/MeMoKBC/pipeline/utils.py:55: SAWarning: Coercing Subquery object into a select() for use in IN(); please pass a select() construct explicitly
  cands = (session.query(candidate_class).filter(candidate_class.id.in_(sub_query)).order_by(candidate_class.id).all())


## LF analysis

NameFull + Abrreviation

In [10]:
LFAnalysis(
    L_train_NFA,
    lfs=sorted(short_long_lfs, key=lambda lf: lf.name)
).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check_all_uppercase_letters,0,[],0.000000,0.000000,0.0
name_full_in_top_percentile_sentence_wise,1,[1],0.267387,0.036739,0.0
name_short_outside_half_percentile_sentence_wise,2,[],0.000000,0.000000,0.0
small_letter_count,3,[1],0.069745,0.052063,0.0
word_count,4,[1],0.098232,0.054224,0.0


NameAbbr + Task

In [11]:
LFAnalysis(
    L_train_NAT,
    lfs=sorted(name_abbr_task_lfs, key=lambda lf: lf.name)
).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
is_medical_abbreviation,0,[],0.000000,0.000000,0.0
lf_length_more_than_three_words,1,[0],0.523618,0.005184,0.0
lf_name_short_in_first_words,2,[0],0.020737,0.005184,0.0
